### pax_icon 생성

In [9]:
# - outputs/{시나리오}/events.json을 읽어
#   pax_icon_{N}.json들을 일괄 생성 (10, 20, 30, 40, 50대 반영)

import os, json
from pathlib import Path
from collections import defaultdict
import pandas as pd

# ===== 사용자 설정 =====
PARQUET_PATH   = "../data/processed/240307_wirye_od.parquet" 
REACT_DATA_DIR = "../visualization/public/data"

# 처리하고 싶은 시나리오와 파일명 라벨 매핑
# (폴더명이 다를 경우 아래 딕셔너리를 수정하세요)
SCENARIOS = {
    "10대": 10,
    "20대": 20,
    "30대": 30,
    "40대": 40,
    "50대": 50,
}

# ---------- 공통 유틸 ----------
def jload(p):
    if not os.path.exists(p): return None
    with open(p, "r", encoding="utf-8") as f:
        try: return json.load(f)
        except:
            f.seek(0); return [json.loads(line) for line in f if line.strip()]

def jdump(obj, p):
    os.makedirs(os.path.dirname(p), exist_ok=True)
    with open(p, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def get_list(obj):
    if obj is None: return []
    if isinstance(obj, list): return obj
    if isinstance(obj, dict):
        for k in ("events","moves"):
            if isinstance(obj.get(k), list): return obj[k]
    return []

def to_seconds_series(s):
    s = pd.to_numeric(s, errors="coerce")
    med = s.dropna().median()
    if pd.isna(med): return s
    if med < 1_000: return s * 60.0   # 분 → 초
    if med > 1e11:  return s / 1000.0 # ms → s
    return s

# ---------- ① 원본 데이터 로드 (고정) ----------
print(f"🔄 원본 데이터 로드 중: {PARQUET_PATH}")
pq = pd.read_parquet(PARQUET_PATH).copy()
pq["KEY1"] = pq["KEY1"].astype(str)

base = pd.DataFrame({
    "pax_ID": pq["KEY1"].astype(str),
    "o_lon": pd.to_numeric(pq["출발_x"], errors="coerce"),
    "o_lat": pd.to_numeric(pq["출발_y"], errors="coerce"),
    "d_lon": pd.to_numeric(pq["도착_x"], errors="coerce"),
    "d_lat": pd.to_numeric(pq["도착_y"], errors="coerce"),
    "t_req": to_seconds_series(pq["승차_timestamp"])
}).drop_duplicates("pax_ID").set_index("pax_ID")

# ---------- ② 시나리오 순회 및 파일 생성 ----------
for folder_name, num_label in SCENARIOS.items():
    scen_dir = Path("../outputs") / folder_name
    ev_path = scen_dir / "events.json"
    
    if not ev_path.exists():
        print(f"⚠️ [SKIP] {folder_name} 시나리오의 events.json이 없습니다.")
        continue
        
    print(f"\n📊 처리 중: {folder_name} (라벨: {num_label})")
    events = get_list(jload(str(ev_path)))
    
    # events에서 정보 추출
    pick_by_id, drop_by_id, veh_by_id = {}, {}, {}
    assign_hist = defaultdict(list)

    for e in events:
        et = str(e.get("type","")).upper()
        rid = e.get("req_id")
        if rid is None: continue
        rid = str(rid)
        
        t = e.get("t") or e.get("ts") or e.get("timestamp") or e.get("time")
        if t is None: continue
        t = float(t)
        
        lon, lat = e.get("lon"), e.get("lat")
        vid = e.get("veh_id")
        att = e.get("attempt")

        if et == "ASSIGN":
            assign_hist[rid].append({"t": t, "attempt": int(att or 1), "veh_id": vid})
            if vid: veh_by_id[rid] = vid
        elif et == "PICKUP":
            if (rid not in pick_by_id) or (t < pick_by_id[rid]["t"]):
                pick_by_id[rid] = {"t": t, "lon": lon, "lat": lat}
                if vid: veh_by_id[rid] = vid
        elif et == "DROPOFF":
            if (rid not in drop_by_id) or (t < drop_by_id[rid]["t"]):
                drop_by_id[rid] = {"t": t, "lon": lon, "lat": lat}

    # rows 생성
    rows = []
    for rid in base.index:
        assigned = (rid in pick_by_id)
        p = pick_by_id.get(rid, {})
        d = drop_by_id.get(rid, {})

        def _get_base(col):
            return float(base.at[rid, col]) if pd.notna(base.at[rid, col]) else 0.0

        plon = float(p.get("lon")) if p.get("lon") is not None else _get_base("o_lon")
        plat = float(p.get("lat")) if p.get("lat") is not None else _get_base("o_lat")
        dlon = float(d.get("lon")) if d.get("lon") is not None else _get_base("d_lon")
        dlat = float(d.get("lat")) if d.get("lat") is not None else _get_base("d_lat")

        hist = sorted(assign_hist.get(rid, []), key=lambda x: x["t"])
        
        rows.append({
            "pax_ID": rid,
            "assigned": bool(assigned),
            "vehicle_id": veh_by_id.get(rid),
            "pickup_location":  [plon, plat],
            "dropoff_location": [dlon, dlat],
            "t_req":  float(base.at[rid, "t_req"]) if pd.notna(base.at[rid, "t_req"]) else None,
            "t_asg":  hist[0]["t"] if hist else None,
            "t_pick": float(p.get("t")) if p.get("t") is not None else None,
            "t_drop": float(d.get("t")) if d.get("t") is not None else None,
            "assign_history": hist,
            "max_attempt": hist[-1]["attempt"] if hist else 0
        })

    out_path = Path(REACT_DATA_DIR) / "pax_icon" / f"pax_icon_{num_label}.json"
    jdump(rows, str(out_path))
    print(f"✅ [SAVE] {out_path} ({len(rows)} pax)")

print("\n✨ 모든 시나리오 데이터 생성이 완료되었습니다!")

🔄 원본 데이터 로드 중: ../data/processed/240307_wirye_od.parquet

📊 처리 중: 10대 (라벨: 10)
✅ [SAVE] ../react-project/public/data/pax_icon/pax_icon_10.json (1366 pax)

📊 처리 중: 20대 (라벨: 20)
✅ [SAVE] ../react-project/public/data/pax_icon/pax_icon_20.json (1366 pax)

📊 처리 중: 30대 (라벨: 30)
✅ [SAVE] ../react-project/public/data/pax_icon/pax_icon_30.json (1366 pax)

📊 처리 중: 40대 (라벨: 40)
✅ [SAVE] ../react-project/public/data/pax_icon/pax_icon_40.json (1366 pax)

📊 처리 중: 50대 (라벨: 50)
✅ [SAVE] ../react-project/public/data/pax_icon/pax_icon_50.json (1366 pax)

✨ 모든 시나리오 데이터 생성이 완료되었습니다!


### trip.json 파일 생성

In [10]:
# moves.json을 OSRM으로 도로 polyline으로 샘플링해 trip_{N}.json들을 일괄 생성
# ⭐️ 도커(OSRM)가 켜져있어야 함

import os, json, math, time
from pathlib import Path
import requests
from collections import defaultdict

# ==== 사용자 설정 ====
REACT_DATA_DIR = "../visualization/public/data"
OSRM_URL = "http://127.0.0.1:8000" 
SAMPLE_EVERY_M = 15.0  # polyline에서 대략 이 간격(미터)로 샘플링 (용량 최적화)

# 처리할 시나리오 매핑 (폴더명: 파일라벨)
SCENARIOS = {
    "10대": 10,
    "20대": 20,
    "30대": 30,
    "40대": 40,
    "50대": 50,
}

# ==== 유틸 ====
def jload(p):
    if not os.path.exists(p): return []
    with open(p, "r", encoding="utf-8") as f:
        try: return json.load(f)
        except:
            f.seek(0); return [json.loads(line) for line in f if line.strip()]

def jdump(obj, p):
    os.makedirs(os.path.dirname(p), exist_ok=True)
    with open(p, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def haversine_m(lon1, lat1, lon2, lat2):
    R = 6371000.0
    dlon, dlat = math.radians(lon2 - lon1), math.radians(lat2 - lat1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(a))

def sample_polyline(coords, every_m=SAMPLE_EVERY_M):
    if not coords: return []
    out = [coords[0]]
    acc = 0.0
    for i in range(1, len(coords)):
        lon1, lat1 = coords[i-1]
        lon2, lat2 = coords[i]
        seg = haversine_m(lon1, lat1, lon2, lat2)
        acc += seg
        if acc >= every_m:
            out.append([lon2, lat2])
            acc = 0.0
    if out[-1] != coords[-1]:
        out.append(coords[-1])
    return out

def osrm_route(lon1, lat1, lon2, lat2, base=OSRM_URL):
    url = f"{base}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    params = {"overview": "full", "geometries": "geojson"}
    try:
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        js = r.json()
        routes = js.get("routes") or []
        if not routes: return [[lon1, lat1], [lon2, lat2]]
        return routes[0]["geometry"]["coordinates"]
    except:
        return [[lon1, lat1], [lon2, lat2]]

# ==== 루프 시작 ====
for folder_name, num_label in SCENARIOS.items():
    scen_dir = Path("../outputs") / folder_name
    moves_path = scen_dir / "moves.json"
    tracks_path = scen_dir / "tracks.json"
    
    if not moves_path.exists():
        print(f"⚠️ [SKIP] {folder_name} 시나리오의 moves.json이 없습니다.")
        continue

    print(f"\n🚗 처리 중: {folder_name} (라벨: {num_label})")
    moves = jload(str(moves_path))
    tracks = jload(str(tracks_path))

    # 차량별 실제 운행 시간 범위 추출 (tracks 기반)
    t_minmax = {}
    for tr in tracks:
        vid = tr.get("veh_id") or tr.get("vehicle_id")
        pts = tr.get("points") or []
        if not vid or not pts: continue
        times = [p.get("t") for p in pts if "t" in p]
        if times: t_minmax[vid] = (min(times), max(times))

    # 차량별 이동 세그먼트 구성
    by_vehicle = defaultdict(list)
    for mv in moves:
        vid = mv.get("veh_id")
        t0, t1 = mv.get("t_start"), mv.get("t_end")
        l1, a1, l2, a2 = mv.get("lon1"), mv.get("lat1"), mv.get("lon2"), mv.get("lat2")
        if vid and isinstance(t0, (int,float)) and None not in (l1, a1, l2, a2):
            by_vehicle[vid].append((t0, t1, l1, a1, l2, a2))

    trip_out = []
    print(f" - 차 대수: {len(by_vehicle)}대")

    for vid, segs in by_vehicle.items():
        segs.sort(key=lambda x: x[0])
        all_coords, all_times = [], []

        for (t0, t1, lon1, lat1, lon2, lat2) in segs:
            # OSRM 도로 경로 탐색 및 샘플링
            coords = sample_polyline(osrm_route(lon1, lat1, lon2, lat2), every_m=SAMPLE_EVERY_M)
            
            # 시간 보간 (거리 비례)
            dists = [0.0]
            for i in range(1, len(coords)):
                dists.append(dists[-1] + haversine_m(coords[i-1][0], coords[i-1][1], coords[i][0], coords[i][1]))
            total = dists[-1] if dists else 0.0
            
            times = []
            for d in dists:
                tau = d / total if total > 0 else 0.0
                times.append(t0 + (t1 - t0) * tau)

            # 이전 세그먼트 끝점과 겹치면 제거
            if all_coords and coords and all_coords[-1] == coords[0]:
                coords, times = coords[1:], times[1:]

            all_coords.extend(coords)
            all_times.extend([int(round(x)) for x in times])

        # 유효한 시간 범위로 클리핑
        tmin, tmax = t_minmax.get(vid, (None, None))
        if tmin is not None and tmax is not None:
            keep = [(c, t) for c, t in zip(all_coords, all_times) if tmin <= t <= tmax]
            if keep:
                all_coords, all_times = [k[0] for k in keep], [k[1] for k in keep]

        if all_coords:
            trip_out.append({"vehicle_id": vid, "trip": all_coords, "timestamp": all_times})

    out_path = Path(REACT_DATA_DIR) / "trip" / f"trip_{num_label}.json"
    jdump(trip_out, str(out_path))
    print(f"✅ [SAVE] {out_path} ({len(trip_out)} vehicles)")

print("\n✨ 모든 Trip 데이터 생성이 완료되었습니다!")


🚗 처리 중: 10대 (라벨: 10)
 - 차 대수: 10대
✅ [SAVE] ../react-project/public/data/trip/trip_10.json (10 vehicles)

🚗 처리 중: 20대 (라벨: 20)
 - 차 대수: 20대
✅ [SAVE] ../react-project/public/data/trip/trip_20.json (20 vehicles)

🚗 처리 중: 30대 (라벨: 30)
 - 차 대수: 30대
✅ [SAVE] ../react-project/public/data/trip/trip_30.json (30 vehicles)

🚗 처리 중: 40대 (라벨: 40)
 - 차 대수: 40대
✅ [SAVE] ../react-project/public/data/trip/trip_40.json (40 vehicles)

🚗 처리 중: 50대 (라벨: 50)
 - 차 대수: 50대
✅ [SAVE] ../react-project/public/data/trip/trip_50.json (50 vehicles)

✨ 모든 Trip 데이터 생성이 완료되었습니다!
